In [ ]:
!nvidia-smi

Wed Dec 13 00:56:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    43W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
!git clone https://github.com/Omid-Nejati/MedViT.git

fatal: destination path 'MedViT' already exists and is not an empty directory.


In [ ]:
cd /content/MedViT

/content/MedViT


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchsummary import summary


##Model

In [ ]:
!pip install timm
!pip install einops

In [ ]:
from MedViT import MedViT_small as tiny

In [ ]:
model = tiny()

initialize_weights...


In [ ]:
model.proj_head[0]

Linear(in_features=1024, out_features=1000, bias=True)

In [ ]:
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=8, bias=True)

In [ ]:
model = model.cuda()

##Dataset

In [ ]:
!pip install medmnist

In [ ]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

In [ ]:
data_flag = 'breastmnist'
# [tissuemnist, pathmnist, chestmnist, dermamnist, octmnist,
# pnemoniamnist, retinamnist, breastmnist, bloodmnist, tissuemnist, organamnist, organcmnist, organsmnist]
download = True

NUM_EPOCHS = 10
BATCH_SIZE = 10
lr = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [ ]:
from torchvision.transforms.transforms import Resize
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import torch.utils.data as data_utils
import os
from PIL import Image
import pandas as pd
from sklearn import metrics
import scipy.stats as stats
import statistics
import numpy as np
input_folder_path_train = "/content/drive/MyDrive/images/preprocessed/train"
output_folder_path_train = "/content/drive/MyDrive/images/preprocessed/train_cropped"
input_folder_path_test = "/content/drive/MyDrive/images/preprocessed/test"
output_folder_path_test = "/content/drive/MyDrive/images/preprocessed/test_cropped"
new_width = 255  # Set the desired width
new_height = 255  # Set the desired height

data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(root=output_folder_path_train, transform=data_transform)
val_dataset = datasets.ImageFolder(root=output_folder_path_test, transform=data_transform)

indices_train = torch.arange(4000)
indices_test = torch.arange(400)

train_dataset_sample = data_utils.Subset(train_dataset,indices_train)
test_dataset_sample = data_utils.Subset(val_dataset,indices_test)

train_loader = DataLoader(train_dataset_sample, batch_size=32, shuffle=True, num_workers=8)
val_loader = DataLoader(test_dataset_sample, batch_size=32, shuffle=True, num_workers=8)


# load the data
# Replace 'your_dataset_directory' with the path to your dataset
train_dataset = datasets.ImageFolder(root=output_folder_path_train, transform=data_transform)
val_dataset = datasets.ImageFolder(root=output_folder_path_test, transform=data_transform)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
print(train_dataset)
print("===================")
print(val_dataset)

Dataset ImageFolder
    Number of datapoints: 9523
    Root location: /content/drive/MyDrive/images/preprocessed/train_cropped
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 497
    Root location: /content/drive/MyDrive/images/preprocessed/test_cropped
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


## Train

In [ ]:
# define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
# train


for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    print('Epoch [%d/%d]'% (epoch+1, NUM_EPOCHS))
    model.train()
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            loss = criterion(outputs, targets)
        else:
            targets = targets.squeeze().long()
            loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

Epoch [1/10]


100%|██████████| 125/125 [02:23<00:00,  1.15s/it]


Epoch [2/10]


100%|██████████| 125/125 [02:14<00:00,  1.08s/it]


Epoch [3/10]


...

100%|██████████| 125/125 [02:14<00:00,  1.08s/it]


Epoch [9/10]


100%|██████████| 125/125 [02:16<00:00,  1.09s/it]


Epoch [10/10]


100%|██████████| 125/125 [02:14<00:00,  1.08s/it]


##Test

In [ ]:
from sklearn import metrics
import scipy.stats as stats
import statistics
import numpy as np

from sklearn.metrics import mean_absolute_error
# evaluation
# Local variables for logging results
best_per_epoch = [[],[]]
best_MAE = 99999    # Set defualt to an artifically high number
best_tau = 0
def test(split):
    model.eval()
    y_true = torch.tensor([]).cuda()
    y_score = torch.tensor([]).cuda()

    data_loader = val_dataset
    correct = 0
    total = 0
    epoch_lowest_MAE = 99999    # Set defualt to an artifically high number
    epoch_log = [[],[]]
    with torch.no_grad():
        all_preds = []
        all_labels = []
        for inputs, targets  in tqdm(val_loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                targets = targets.squeeze().long()
                outputs = outputs.softmax(dim=-1)
                targets = targets.float().resize_(len(targets), 1)


            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

            inputs, targets = inputs.to('cpu'), targets.to('cpu')
            outputs = outputs.to('cpu')

            [all_preds.append(torch.argmax(i).item()) for i in outputs]
            [all_labels.append(i.item()) for i in targets]

            total += outputs.size(0)
            correct += (outputs == targets).sum().item()

        y_true = y_true.cpu().numpy()
        y_score = y_score.detach().cpu().numpy()

        #evaluator = Evaluator(data_flag, split)
        #metrics = evaluator.evaluate(y_score)

        #print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))
        print(all_preds, all_labels)
        #all_preds = all_preds.numpy()
        #all_labels = all_labels.numpy()
        tau, p_value = stats.kendalltau(all_preds, all_labels)
        epoch_Tau = tau

        # Calc mae
        label_set = list(set(all_labels))
        all_mae = []
        for label in label_set:
            index_list = [i for i, x in enumerate(all_labels) if x == label]
            pred_list = [all_preds[i] for i in index_list]
            label_list = [all_labels[i] for i in index_list]
            mae = mean_absolute_error(pred_list, label_list)
            all_mae.append(mae)
        epoch_MAE = np.average(all_mae)

        accuracy = correct / total
        print(f"Validation Accuracy: {accuracy * 100:.2f}%")

        epoch_log[0].append(epoch_MAE)
        epoch_log[1].append(epoch_Tau)


    MAE_average = statistics.mean(epoch_log[0])
    Tau_average = statistics.mean(epoch_log[1])

    MAE_deviation = statistics.pstdev(epoch_log[0])
    Tau_deviation = statistics.pstdev(epoch_log[1])


    print(f"AMAE value: {MAE_average} +- {MAE_deviation}")
    print(f"TAU value: {Tau_average} +- {Tau_deviation}")


print('==> Evaluating ...')
test('test')

==> Evaluating ...


100%|██████████| 13/13 [00:06<00:00,  1.96it/s]

[3, 1, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 3, 3, 2, 3, 1, 1, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 2, 3, 2, 1, 0, 1, 3, 3, 3, 1, 3, 1, 3, 1, 3, 3, 3, 3, 0, 3, 3, 2, 0, 3, 1, 3, 1, 1, 0, 1, 1, 3, 1, 3, 3, 3, 2, 1, 1, 2, 1, 2, 1, 1, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 1, 3, 1, 3, 1, 0, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 0, 1, 0, 1, 3, 1, 3, 3, 1, 3, 3, 3, 1, 3, 3, 3, 1, 1, 0, 3, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 3, 1, 3, 1, 1, 1, 3, 0, 3, 0, 3, 3, 3, 1, 3, 1, 1, 1, 3, 1, 3, 1, 3, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3, 0, 3, 1, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 3, 3, 2, 2, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 0, 3, 1, 3, 1, 3, 1, 3, 2, 1, 3, 1, 1, 2, 3, 3, 1, 0, 3, 3, 1, 3, 1, 0, 3, 2, 3, 3, 3, 1, 1, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 1, 2, 3, 3, 1, 3, 3, 3, 1, 3, 2, 3, 1, 3, 0, 3, 3, 2, 1, 3, 3, 1, 3, 0, 1, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 2, 3, 3, 